# Analytic Modeling of the Recurrent Outbreak of the Novel Coronavirus Disease 2019 Epidemics

**Description.** This report is an attempt at analytical modeling of repeated outbreaks of the Coronavirus Disease 2019 (COVID-19) epidemic for the use case of The Republic of Croatia. The idea of the mentioned approach lies in observing the epidemic and the sudden jumps and falls of the total number of actively infected individuals as a series of *pulses* which represent the current presence of the virus in the population, conditioned by numerous factors. Results of the report show the direct dependence of the number of actively infected individuals on the effective reproduction number immediately before. Also, an exceptional fit of the dynamics of the number of active cases with the series of Heidler functions is achived through CoroPy, Python package for COVID-19 epidemics modeling.

In [ ]:
import os
import datetime as dt

from coropy.growth_models import GrowthCOVIDModel
from coropy.compartmental_models import SEIRDModel
import coropy.reproduction_rate as R
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

In [ ]:
# configure plotting
utils.configure(grid=False)
hex_blue = utils.default_colors('blue')
hex_red = utils.default_colors('red')
hex_purple = utils.default_colors('purple')

In [ ]:
def extract_active_cases(start, end, country):
    cases = pd.read_csv(os.path.join(os.pardir, 'data', 'international_cases.csv'))
    cases['Date'] = pd.to_datetime(cases['Date'])
    cases_per_country = cases[(cases['Country'] == country)
                              & (cases['Date'] >= start)
                              & (cases['Date'] <= end)]
    cc = cases_per_country['Confirmed'].to_numpy()
    rc = cases_per_country['Recovered'].to_numpy()
    dc = cases_per_country['Deaths'].to_numpy()
    active = cc - rc - dc
    return active, cc, rc, dc

In [ ]:
start_date = dt.datetime(2020, 2, 25)
end_date = dt.datetime(2021, 7, 2)
active, cum_positives, cum_recovered, cum_deceased = extract_active_cases(start_date, end_date, 'Croatia')

In [ ]:
# start of different epi waves
eff_dates=[dt.datetime(2020, 2, 25), dt.datetime(2020, 6, 1), dt.datetime(2020, 8, 1),  # initial outbreak and first wave
           dt.datetime(2020, 10, 1),  # second wave
           dt.datetime(2021, 2, 15)]  # third wave

## 1 Heidler function

In [ ]:
def heidler(t, t1, t2, n, I0):
    xn = np.sign(t / t1) * np.abs(t / t1) ** n
    x = xn / (1 + xn)
    y = np.exp(-t / t2)
    return I0 * x * y

In [ ]:
X = []
Y = []

# past wave(s)
start_idx = 0
i = 0
amplitude = 0
for wave_start, wave_end in zip(eff_dates[:-1], eff_dates[1:]):
    end_idx = start_idx+abs((wave_end - wave_start).days)
    model = GrowthCOVIDModel(heidler, normalize=True, calc_ci=False)
    model.fit(active[start_idx:end_idx])
    x, fitted = model.get_fitted
    X.extend(x)
    Y.extend(amplitude + fitted)
    if i == 3:
        amplitude += fitted[-1]
    start_idx = end_idx  # update indexing
    i += 1

# current wave   
model = GrowthCOVIDModel(heidler, normalize=True, calc_ci=False)
model.fit(active[start_idx:])
x, fitted = model.get_fitted
X.extend(x)
Y.extend(amplitude - fitted[0] + fitted)

In [ ]:
# goodness-of-fit
sre_fit = np.sqrt(np.mean((active - Y)**2))
sre_fit

In [ ]:
# dates
dates = matplotlib.dates.drange(start_date, end_date + dt.timedelta(1), dt.timedelta(days=1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(dates, active, marker='o', markersize=4, linestyle='none', c=hex_purple, markevery=4, label='$I[t]$')
ax1.plot(dates, Y, c=hex_blue, label=f'$\\hat I(t)$')
ax1.tick_params(axis='y', labelcolor=hex_blue)
ax1.set_ylabel('$N$', color=hex_blue)
ax1.set_yticks([0, 10000, 20000, 30000])
ax1.set_ylim([-2500, 30000])
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.plot(dates[:-15-3+1][::12], R._estimate(cum_positives, 15, 3)[::12], color='gray', linestyle='-', marker='s', markersize=3, label='$R_t$(t)')
ax2.axhline(y=1, linestyle='--', linewidth=1, c='gray')
ax2.tick_params(axis='y', labelcolor='gray')
ax2.set_ylabel('$R_t$', color='gray')
ax2.set_yticks([0, 1, 2, 3])
ax2.set_ylim([-0.25, 3])
ax2.legend(loc='upper right')

fig.tight_layout()
_ = fig.gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y/%m/%d'))
_ = fig.gca().xaxis.set_major_locator(matplotlib.dates.MonthLocator(bymonth=[1, 4, 7, 10]))
_ = plt.gcf().autofmt_xdate()
plt.show()

fname = f'multiwave_Heidler'
fig.savefig(fname=os.path.join(os.pardir, 'figures', fname + '.png'),
            format='png', bbox_inches='tight', dpi=1000)

## 2 SEIRD model

In [ ]:
start_date = dt.datetime(2020, 2, 25)
end_date = dt.datetime(2021, 7, 1)
active, cum_positives, cum_recovered, cum_deceased = extract_active_cases(start_date, end_date, 'Croatia')

# start of different epi waves
eff_dates=[dt.datetime(2020, 2, 25), dt.datetime(2020, 6, 6), dt.datetime(2020, 8, 8),  # initial outbreak and first wave
           dt.datetime(2020, 10, 10),  # second wave
           dt.datetime(2021, 2, 20)]  # third wave

# dates
dates = matplotlib.dates.drange(start_date, end_date + dt.timedelta(1), dt.timedelta(days=1))

In [ ]:
eff_population_scaler = 1
first_wave_eff_population = 2200
S0 = first_wave_eff_population * eff_population_scaler
E0 = 3 * cum_positives[0]
I0 = cum_positives[0]
R0 = cum_recovered[0]
D0 = cum_deceased[0]
IC = (S0, E0, I0, R0, D0)

S_tot, E_tot, I_tot, R_tot, D_tot = [], [], [], [], []

# past wave(s)
start_idx = 0
i = 0
for start_date, end_date in zip(eff_dates[:-1], eff_dates[1:]):
    end_idx = start_idx+abs((end_date - start_date).days)
    model = SEIRDModel()
    _, _ = model.fit(cum_positives[start_idx:end_idx],
                     cum_recovered[start_idx:end_idx],
                     cum_deceased[start_idx:end_idx],
                     IC)
    (S, E, I, R, D) = model.simulate()
    S_tot.extend(S.tolist())
    E_tot.extend(E.tolist())
    I_tot.extend(I.tolist())
    R_tot.extend(R.tolist())
    D_tot.extend(D.tolist())
    
    if i < 2:
        eff_population_scaler += 1
    else:
        eff_population_scaler += 10
    S0 = S0 * eff_population_scaler
    IC = (S0, 5 * I[-1], I[-1], R[-1], D[-1])  # update initial conditions
    start_idx = end_idx  # update indexing
    i += 1

# current wave   
model = SEIRDModel()
IC = (0.07 * S0, 5 * I[-1], I[-1], R[-1], D[-1])
_, _ = model.fit(cum_positives[start_idx:],
                 cum_recovered[start_idx:],
                 cum_deceased[start_idx:],
                 IC)
(S, E, I, R, D) = model.simulate()
S_tot.extend(S.tolist())
E_tot.extend(E.tolist())
I_tot.extend(I.tolist())
R_tot.extend(R.tolist())
D_tot.extend(D.tolist())

In [ ]:
# goodness-of-fit
sre_fit = np.sqrt(np.mean((active - I_tot)**2))
sre_fit

In [ ]:
# visualize
fig, ax = plt.subplots()
ax.plot(dates, active, marker='o', markersize=4, linestyle='none', c=hex_purple, markevery=3, label='$I[t]$')
ax.plot(dates, I_tot, c=hex_blue, label=f'$\\hat I(t)$')
ax.tick_params(axis='y')
ax.set_ylabel('$N$')
ax.set_yticks([0, 10000, 20000, 30000])
ax.set_ylim([-3000, 35000])
ax.legend(loc='upper left')
fig.tight_layout()

_ = fig.gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y/%m/%d'))
_ = fig.gca().xaxis.set_major_locator(matplotlib.dates.MonthLocator(bymonth=[1, 4, 7, 10]))
_ = plt.gcf().autofmt_xdate()
plt.show()

fname = f'multiwave_SEIRD'
fig.savefig(fname=os.path.join(os.pardir, 'figures', fname + '.png'),
            format='png', bbox_inches='tight', dpi=1000)